### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import urllib
import matplotlib.dates as mdates

### Loading the data in a dataframe

In [32]:
df = pd.read_csv('data.csv')
pd.options.display.max_columns = 999
df

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
0,1,2,1,0,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,1,2,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,1,2,1,2,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
3,1,2,1,1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,1,2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,2,1,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
6,1,2,1,1,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
8,1,2,1,1,3,4,4,4,4,4,4,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
9,1,2,1,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


### Reorganizing the data - Difficulty values
We want to change the 15 scale to -2/2 so that we can have 0 as an absolute value

In [33]:
df['difficulty'] = df['difficulty'].apply(lambda x: x - 3)

### Assessing the class size - Grouping the 13 classes

In [35]:
df.rename(columns={'nb.repeat':'no.taken'}, inplace=True)

In [36]:
# spc means students per class
spc = df.groupby(['class', 'no.taken']).size()
spc

class  no.taken
1      1           272
       2            25
       3             6
2      1           130
       2             7
       3             3
3      1           778
       2            87
       3            39
4      1           154
       2            20
       3            13
5      1           611
       2            28
       3            17
6      1           494
       2            55
       3             9
7      1           150
       2            14
       3            23
8      1           379
       2            69
       3            52
9      1           400
       2            99
       3            72
10     1           425
       2            14
       3             9
11     1           431
       2            43
       3            10
12     1            34
       2             5
       3             2
13     1           651
       2           110
       3            80
dtype: int64

### Renaming the nb.repeats column name to no.taken 
The name is confusing and can be interpreted that a student has taken the calss already once and this
value represents the number of times that he repeated the class

### Assessing how many students have taken the class and how many times

In [37]:
df.groupby('no.taken').size()

no.taken
1    4909
2     576
3     335
dtype: int64

### Checking how many students have taken the class how many times per class

In [38]:
# sapc means student attempts per class
sapc = df.groupby(['class', 'no.taken']).size()
sapc

class  no.taken
1      1           272
       2            25
       3             6
2      1           130
       2             7
       3             3
3      1           778
       2            87
       3            39
4      1           154
       2            20
       3            13
5      1           611
       2            28
       3            17
6      1           494
       2            55
       3             9
7      1           150
       2            14
       3            23
8      1           379
       2            69
       3            52
9      1           400
       2            99
       3            72
10     1           425
       2            14
       3             9
11     1           431
       2            43
       3            10
12     1            34
       2             5
       3             2
13     1           651
       2           110
       3            80
dtype: int64

### Converting the values in percentages to better understand the dataset

In [39]:
# sapc means student attempts per class percentage
sapcp = df.groupby(['class', 'no.taken']).size().reset_index(name='count')
sapcp['Percentage'] = sapcp.groupby('class')['count'].transform(lambda x: x/sum(x) * 100)
sapcp

,class,no.taken,count,Percentage
0,1,1,272,89.768977
1,1,2,25,8.250825
2,1,3,6,1.980198
3,2,1,130,92.857143
4,2,2,7,5.000000
5,2,3,3,2.142857
6,3,1,778,86.061947
7,3,2,87,9.623894
8,3,3,39,4.314159
9,4,1,154,82.352941


### Getting the mean of the percentages from 1, 2, 3 attemps

In [40]:
sapcp.groupby('no.taken').mean()['Percentage']

no.taken
1    84.848346
2     9.507965
3     5.643689
Name: Percentage, dtype: float64

Based on the previous block, we can assume that the teachers had little to do with students passing/failing.
By calculating the mean for the atempts, we can see that 84.8% of students graduated in the first try, 9.5% in the second try and 5.6% in the third try 

### Split the data in chunks - classes to prepare it for further analysis
This way, we can quickly access the data by class

Data can be accessed like so dfdc[class]

In [41]:
# Create a list of classes
classes = df['class'].unique()
classes
# Create a data frame dictionary to store the entries per class for quick access
# dfd is dataframe dictionary
dfdc = {elem : pd.DataFrame for elem in classes}
for key in dfdc.keys():
    dfdc[key] = df[:][df['class'] == key]


### Split the data in chunks - teachers to prepare it for further analysis
This way, we can quickly access the entries relating to a teacher

Data can be accessed like so dfdt[teacher]

In [30]:
# Create a list of teachers
teachers = df['instr'].unique()
teachers
# Create a data frame dictionary to store the entries per teacher for quick access
# dfdp is dataframe dictionary
dfdp = {elem : pd.DataFrame for elem in teachers}
for key in dfdc.keys():
    dfdp[key] = df[:][df['instr'] == key]

,instr,class,no.taken,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
0,1,2,1,0,1,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,1,2,1,1,0,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,2,1,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
3,1,2,1,1,0,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,1,2,1,0,-2,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,1,2,1,3,0,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
6,1,2,1,1,0,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,0,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
8,1,2,1,1,0,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
9,1,2,1,4,1,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


### Important observation

It seems that there are a lot of entries that have the same score in all the question

This might mean that the students have untruthfully graded the questions due to laziness or other factors

Though the factors are not important, we want to assess what is the percentage of entries that have the difference between the entry itself and the mode of the scores of the question 0. At a later date we will determine whether such entries should be treated differently or not.

In [53]:
# First step is to create a dataset that holds only the questions
# qdf represents questions dataframe
qdf = df[df.columns.difference(['instr', 'class', 'no.taken', 'attendance', 'difficulty'])]
qdf

,Q1,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q2,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q3,Q4,Q5,Q6,Q7,Q8,Q9
0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
6,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
7,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
8,4,4,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4
9,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4


In [56]:
# Second step is to calculate the median of the questions per row
qdf['median'] = qdf.median(1)
qdf

/Users/_bogz_/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Q1,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q2,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q3,Q4,Q5,Q6,Q7,Q8,Q9,median
0,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0
1,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0
2,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5.0
3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3.0
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0
5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4.0
6,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4.0
7,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5.0
8,4,4,5,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4.0
9,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4.0


In [57]:
# Third step is to make the difference between each value of the questions and the median
qdf = qdf.apply(lambda x: x - x['median'], axis=1)
qdf

,Q1,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q2,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q3,Q4,Q5,Q6,Q7,Q8,Q9,median
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# Fourth step is to count how many of such records where the values of questions after the applied difference is 0
qtc[qtc.apply(pd.value_counts, 1).isna().any(axis=1)]
# df1 = df[df.isna().any(axis=1)]

,Q1,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Q2,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28,Q3,Q4,Q5,Q6,Q7,Q8,Q9,median
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
10,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


In [75]:
100 *2985/5819 

51.297473792747894

In [ ]:
## 51% of the values are all equal. What do we do now...